In [1]:
import pandas as pd
import numpy as np
import random

In [35]:
n_cells = 100

In [ ]:
cell_ids = {'ANF': [i for i in range(n_cells)],
            'I2': [i for i in range(n_cells)],
            'P': [i for i in range(n_cells)]}

num_octaves = 4

bw_octs = {'ANF': 0.4, 'I2': 0.6, 'W': 0.1}
bw_nums = {'ANF': int(bw_octs['ANF'] / (num_octaves/n_cells)),
           'I2': int(bw_octs['I2'] / (num_octaves/n_cells)),
           'W': int(bw_octs['W'] / (num_octaves/n_cells))}
n_scales = {'ANF': 0.6,
            'I2': 0.175,
            'W': 0.75}
ns = {'ANF': int(np.floor(n_scales['ANF']*bw_nums['ANF'])), 
      'I2': int(np.floor(n_scales['I2']*bw_nums['I2'])),
      'W': int(np.floor(n_scales['W']*bw_nums['W']))}

conns_list = {'ANF_I2': [],
              'ANF_P': [],
              'I2_P': []}


In [40]:
[4*i for i in [0.4, 0.6]] #, 0.1]]

[1.6, 2.4]

In [37]:
bw_nums, ns

({'ANF': 10, 'I2': 15, 'W': 2}, {'ANF': 6, 'I2': 2, 'W': 1})

In [ ]:
for conn in conns_list.keys():

    source = conn.split('_')[0]
    source_ids = cell_ids[source]

    target = conn.split('_')[1]
    target_ids = cell_ids[target]

    bw = bw_nums[source]
    if bw == 1: bw += 1
    bw_split = bw//2

    conn_list = []
    
    for target_id in target_ids:

        n_source = ns[source]
        if n_source == 0: n_source += 1

        lb = target_id - bw_split
        if lb < 0: lb = 0
        ub = target_id + bw_split+1

        source_pool = source_ids[lb:ub]

        # if n_source > bw/len(source_pool):
        n_source *= (len(source_pool)/bw)

        source_rand = random.sample(source_pool, int(np.floor(n_source)))

        conn_list.extend([[i, target_id] for i in source_rand])
    
    conns_list[conn] = conn_list
